# Finetuning using Huggingface e2e

In [9]:
import os

import pandas as pd
import numpy as np
import torch

import transformers

#from transformers import  BertModel, BertTokenizer, BertForSequenceClassification, AdamW
from transformers import (AutoModel, AutoTokenizer, AutoConfig, AutoModelForSequenceClassification,
                          Trainer, TrainingArguments)


In [10]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

In [11]:
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


## Load processed labeled data

In [12]:
MAX_LENGHT = 512
DATA_DIR = "../../../data/merged_split/"

checkpoint = "dbmdz/bert-base-historic-dutch-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

loading configuration file https://huggingface.co/dbmdz/bert-base-historic-dutch-cased/resolve/main/config.json from cache at /Users/Zahed001/.cache/huggingface/transformers/1dd6939500d72963b7c7ed7033ba4857c1576a28f9f2819d8dde7a71f02e879e.5ffd597ad0cae5679a9afabdf80a381f3c67384ce1edbb6f01a13a7730a45b2e
Model config BertConfig {
  "_name_or_path": "dbmdz/bert-base-historic-dutch-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.21.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}

loading file https://huggingface.co/dbmdz/bert-base

### Helper: tokenizer

In [13]:
from transformers import DataCollatorWithPadding

def tokenize_function(row):
    return tokenizer(
        row["text"],
        truncation=True,
)

def tokenize_data(raw_datasets):
    tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
    return tokenized_datasets

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, max_length=MAX_LENGHT)

# Fine tune multiple models

## Finetune: Type per decade (12 models)

In [10]:
%%time 

import os
import datasets
from datasets import Dataset
from sklearn.model_selection import train_test_split
from transformers import EarlyStoppingCallback


DECADES = ["1960s", "1970s", "1980s", "1990s"]
TYPES = ["oil", "gas", "coal"]

for DECADE in DECADES:
    for TYPE in TYPES:
        # Load dataset
        NAME = f"{DECADE}_{TYPE}"
        
        try:
            os.mkdir(f"../../../output/bert_finetune/{NAME}")
        except:
            continue
        
        DIR_MODEL = f"../../../output/bert_finetune/{NAME}"
        
        df = pd.read_csv(os.path.join(DATA_DIR,DECADE, f"{DECADE}_{TYPE}_merged_split.csv")).sample(frac=1)
        # Create training, validation and test
        # Divide into train and val
        train_texts, val_texts, train_labels, val_labels = train_test_split(list(df.text_split), list(df.labels), test_size=.2)
        train_dataset = pd.DataFrame()
        train_dataset["text"] = train_texts
        train_dataset["labels"] = train_labels

        # Divide val into val and test"
        print(" Divide val into val and test")
        test_texts, val_texts, test_labels, val_labels = train_test_split(val_texts, val_labels, test_size=.5)
        validation_dataset = pd.DataFrame()
        validation_dataset["text"] = val_texts
        validation_dataset["labels"] = val_labels

        test_dataset = pd.DataFrame()
        test_dataset["text"] = test_texts
        test_dataset["labels"] = test_labels
        
        print("Transform into Dataset from hf")
        # Transform into Dataset from hf
        train_dataset_raw = Dataset.from_pandas(train_dataset)
        train_dataset_raw = train_dataset_raw.rename_column('labels', 'label')

        validation_dataset_raw = Dataset.from_pandas(validation_dataset)
        validation_dataset_raw = validation_dataset_raw.rename_column('labels', 'label')

        test_dataset_raw = Dataset.from_pandas(test_dataset)
        test_dataset_raw = test_dataset_raw.rename_column('labels', 'label')
        
        print("Tokenize the datasets")
        # Tokenize the datasets
        training_dataset_tokenized = tokenize_data(train_dataset_raw)
        validation_dataset_tokenized = tokenize_data(validation_dataset_raw)
        test_dataset_tokenized = tokenize_data(test_dataset_raw)
        
        print("Save to disk")
        # Save to disk
        os.mkdir(os.path.join(DIR_MODEL, "datasets"))
        DIR_MODEL_DATASET = os.path.join(DIR_MODEL, "datasets")

        os.mkdir(os.path.join(DIR_MODEL_DATASET, "training"))
        training_dataset_tokenized.save_to_disk(os.path.join(DIR_MODEL_DATASET, "training"))

        os.mkdir(os.path.join(DIR_MODEL_DATASET, "validation"))
        validation_dataset_tokenized.save_to_disk(os.path.join(DIR_MODEL_DATASET, "validation"))

        os.mkdir(os.path.join(DIR_MODEL_DATASET, "test"))
        test_dataset_tokenized.save_to_disk(os.path.join(DIR_MODEL_DATASET, "test"))

        
        print("Create data collator")
        # Create data collator
        data_collator = DataCollatorWithPadding(tokenizer=tokenizer, max_length=MAX_LENGHT)
        
        training_args = TrainingArguments(
            output_dir = DIR_MODEL,
            num_train_epochs=4,              # total number of training epochs
            evaluation_strategy="steps",
            logging_dir="../../../output/bert_finetune/logging",
            load_best_model_at_end=True,  
            seed=2020,
            #label_names=["label"], # check this
            disable_tqdm=False
        )

        model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)
        
        trainer = Trainer(
            model,
            training_args,
            train_dataset=training_dataset_tokenized,
            eval_dataset=validation_dataset_tokenized,
            data_collator=data_collator,
            tokenizer=tokenizer,
        )

        trainer.train()
        
        model.save_pretrained(DIR_MODEL)
        

 Divide val into val and test
Transform into Dataset from hf
Tokenize the datasets


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 93.75ba/s]


Save to disk
Create data collator


Some weights of the model checkpoint at dbmdz/bert-base-historic-dutch-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassificatio

Step,Training Loss,Validation Loss
500,1.059200,0.966065
1000,0.624800,1.192456


...

Save to disk
Create data collator


loading configuration file https://huggingface.co/dbmdz/bert-base-historic-dutch-cased/resolve/main/config.json from cache at /Users/Zahed001/.cache/huggingface/transformers/1dd6939500d72963b7c7ed7033ba4857c1576a28f9f2819d8dde7a71f02e879e.5ffd597ad0cae5679a9afabdf80a381f3c67384ce1edbb6f01a13a7730a45b2e
Model config BertConfig {
  "_name_or_path": "dbmdz/bert-base-historic-dutch-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "

Step,Training Loss,Validation Loss
500,0.782400,0.899392


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 134
  Batch size = 8
Saving model checkpoint to ../../../output/bert_finetune/1990s_gas/checkpoint-500
Configuration saved in ../../../output/bert_finetune/1990s_gas/checkpoint-500/config.json


RuntimeError: [enforce fail at inline_container.cc:319] . unexpected pos 433093632 vs 433093520

## Finetune: Only types (3 models)

In [9]:
%%time 

import os
import datasets
from datasets import Dataset
from sklearn.model_selection import train_test_split
from transformers import EarlyStoppingCallback


DECADES = ["1960s", "1970s", "1980s", "1990s"]
TYPES = ["oil"]

for TYPE in TYPES:
    # Load dataset
    NAME = f"{TYPE}"
    
    try:
        os.mkdir(f"/home/leonardovida/data/volume_1/delphbert-results/6-finetuning-outputs/model-per-type/{NAME}")
    except :
        continue
    
    DIR_MODEL = f"/home/leonardovida/data/volume_1/delphbert-results/6-finetuning-outputs/model-per-type/{NAME}"
    df_list = []
    for DECADE in DECADES:
        temp = pd.read_csv(os.path.join(DATA_DIR, f"{DECADE}_{TYPE}_merged_split.csv"))
        df_list.append(temp)
    df = pd.concat(df_list)

    # Create training, validation and test
    # Divide into train and val
    train_texts, val_texts, train_labels, val_labels = train_test_split(list(df.text_split), list(df.labels), test_size=.2)
    train_dataset = pd.DataFrame()
    train_dataset["text"] = train_texts
    train_dataset["labels"] = train_labels

    # Divide val into val and test
    test_texts, val_texts, test_labels, val_labels = train_test_split(val_texts, val_labels, test_size=.5)
    validation_dataset = pd.DataFrame()
    validation_dataset["text"] = val_texts
    validation_dataset["labels"] = val_labels

    test_dataset = pd.DataFrame()
    test_dataset["text"] = test_texts
    test_dataset["labels"] = test_labels

    # Transform into Dataset from hf
    train_dataset_raw = Dataset.from_pandas(train_dataset)
    train_dataset_raw = train_dataset_raw.rename_column('labels', 'label')

    validation_dataset_raw = Dataset.from_pandas(validation_dataset)
    validation_dataset_raw = validation_dataset_raw.rename_column('labels', 'label')

    test_dataset_raw = Dataset.from_pandas(test_dataset)
    test_dataset_raw = test_dataset_raw.rename_column('labels', 'label')

    # Tokenize the datasets
    training_dataset_tokenized = tokenize_data(train_dataset_raw)
    validation_dataset_tokenized = tokenize_data(validation_dataset_raw)
    test_dataset_tokenized = tokenize_data(test_dataset_raw)

    # Save to disk
    os.mkdir(os.path.join(DIR_MODEL, "datasets"))
    DIR_MODEL_DATASET = os.path.join(DIR_MODEL, "datasets")
    
    os.mkdir(os.path.join(DIR_MODEL_DATASET, "training"))
    training_dataset_tokenized.save_to_disk(os.path.join(DIR_MODEL_DATASET, "training"))

    os.mkdir(os.path.join(DIR_MODEL_DATASET, "validation"))
    validation_dataset_tokenized.save_to_disk(os.path.join(DIR_MODEL_DATASET, "validation"))

    os.mkdir(os.path.join(DIR_MODEL_DATASET, "test"))
    test_dataset_tokenized.save_to_disk(os.path.join(DIR_MODEL_DATASET, "test"))

    # Create data collator
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer, max_length=MAX_LENGHT)

    training_args = TrainingArguments(
        output_dir = DIR_MODEL,
        num_train_epochs=4,              # total number of training epochs
        evaluation_strategy="steps",
        logging_dir="/home/leonardovida/data/volume_1/delphbert-results/6-finetuning-outputs/model-per-type/logging",
        load_best_model_at_end=True,  
        seed=2020,
        #label_names=["label"], # check this
        disable_tqdm=False
    )

    model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)

    trainer = Trainer(
        model,
        training_args,
        train_dataset=training_dataset_tokenized,
        eval_dataset=validation_dataset_tokenized,
        data_collator=data_collator,
        tokenizer=tokenizer,
    )

    trainer.train()

    model.save_pretrained(DIR_MODEL)



Some weights of the model checkpoint at wietsedv/bert-base-dutch-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at wietsedv/bert-

Step,Training Loss,Validation Loss,Runtime,Samples Per Second
500,1.024500,0.962023,4.517700,384.046000
1000,0.950900,0.906383,4.440700,390.704000
1500,0.924500,0.935806,4.476300,387.595000
2000,0.787800,0.952080,4.504800,385.144000
2500,0.655000,0.921921,4.537000,382.412000


CPU times: user 6min 29s, sys: 2min 9s, total: 8min 38s
Wall time: 5min 38s


## Test models

In [17]:
%%time 

import os
import glob
import datasets
from datasets import Dataset
from sklearn.model_selection import train_test_split
from transformers import EarlyStoppingCallback


DECADES = ["1960s","1970s","1980s", "1990s"]
print(DECADES)

for DECADE in DECADES:
    # Load dataset
    NAME = f"{DECADE}"
    
    try:
        os.mkdir(f"../../../output/bert_finetune/{NAME}")
    except :
        continue
    
    DIR_MODEL = f"../../../output/bert_finetune/{NAME}"
    
    DATA_DIR = "../../../data/merged_split/"
    FILE_DIR = glob.glob(os.path.join(DATA_DIR,DECADE,'*.csv'))

    
    df_list = [] 
    for f in FILE_DIR:
        df_tmp = pd.read_csv(f)
        df_list.append(df_tmp)
    df = pd.concat(df_list)

    print('decade:',NAME)
    print('df size :', df.shape)
    
    # Create training, validation and test
    # Divide into train and val
    train_texts, val_texts, train_labels, val_labels = train_test_split(list(df.text_split), list(df.labels), test_size=.2)
    train_dataset = pd.DataFrame()
    train_dataset["text"] = train_texts
    train_dataset["labels"] = train_labels

    # Divide val into val and test
    test_texts, val_texts, test_labels, val_labels = train_test_split(val_texts, val_labels, test_size=.5)
    validation_dataset = pd.DataFrame()
    validation_dataset["text"] = val_texts
    validation_dataset["labels"] = val_labels

    test_dataset = pd.DataFrame()
    test_dataset["text"] = test_texts
    test_dataset["labels"] = test_labels

    print('train_dataset:', train_dataset.shape)
    print('validation_dataset:', validation_dataset.shape)
    print('test_dataset:', test_dataset.shape)
    # Transform into Dataset from hf
    train_dataset_raw = Dataset.from_pandas(train_dataset)
    train_dataset_raw = train_dataset_raw.rename_column('labels', 'label')

    validation_dataset_raw = Dataset.from_pandas(validation_dataset)
    validation_dataset_raw = validation_dataset_raw.rename_column('labels', 'label')

    test_dataset_raw = Dataset.from_pandas(test_dataset)
    test_dataset_raw = test_dataset_raw.rename_column('labels', 'label')

    # Tokenize the datasets
    training_dataset_tokenized = tokenize_data(train_dataset_raw)
    validation_dataset_tokenized = tokenize_data(validation_dataset_raw)
    test_dataset_tokenized = tokenize_data(test_dataset_raw)

    # Save to disk
    os.mkdir(os.path.join(DIR_MODEL, "datasets"))
    DIR_MODEL_DATASET = os.path.join(DIR_MODEL, "datasets")
    
    os.mkdir(os.path.join(DIR_MODEL_DATASET, "training"))
    training_dataset_tokenized.save_to_disk(os.path.join(DIR_MODEL_DATASET, "training"))

    os.mkdir(os.path.join(DIR_MODEL_DATASET, "validation"))
    validation_dataset_tokenized.save_to_disk(os.path.join(DIR_MODEL_DATASET, "validation"))

    os.mkdir(os.path.join(DIR_MODEL_DATASET, "test"))
    test_dataset_tokenized.save_to_disk(os.path.join(DIR_MODEL_DATASET, "test"))

    # Create data collator
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer, max_length=MAX_LENGHT)

    training_args = TrainingArguments(
        output_dir = DIR_MODEL,
        num_train_epochs=4,              # total number of training epochs
        evaluation_strategy="steps",
        logging_dir= os.path.join(DIR_MODEL, "logging"),
        load_best_model_at_end=True,  
        seed=2020,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        #label_names=["label"], # check this
        disable_tqdm=False
    )

    model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)

    trainer = Trainer(
        model,
        training_args,
        train_dataset=training_dataset_tokenized,
        eval_dataset=validation_dataset_tokenized,
        data_collator=data_collator,
        tokenizer=tokenizer,
    )

    trainer.train()

    model.save_pretrained(DIR_MODEL)


['1960s', '1970s', '1980s', '1990s']
decade: 1960s
df size : (10391, 6)
train_dataset: (8312, 2)
validation_dataset: (1040, 2)
test_dataset: (1039, 2)


100%|█████████████████████████████████████████████| 2/2 [00:00<00:00, 75.99ba/s]
using `logging_steps` to initialize `eval_steps` to 500
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file https://huggingface.co/dbmdz/bert-base-historic-dutch-cased/resolve/main/config.json from cache at /Users/Zahed001/.cache/huggingface/transformers/1dd6939500d72963b7c7ed7033ba4857c1576a28f9f2819d8dde7a71f02e879e.5ffd597ad0cae5679a9afabdf80a381f3c67384ce1edbb6f01a13a7730a45b2e
Model config BertConfig {
  "_name_or_path": "dbmdz/bert-base-historic-dutch-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidd

Step,Training Loss,Validation Loss
500,1.053400,1.032067
1000,1.045000,1.007771
1500,1.041800,1.007202
2000,1.032700,1.010823


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1040
  Batch size = 16
Saving model checkpoint to ../../../output/bert_finetune/1960s/checkpoint-500
Configuration saved in ../../../output/bert_finetune/1960s/checkpoint-500/config.json
Model weights saved in ../../../output/bert_finetune/1960s/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ../../../output/bert_finetune/1960s/checkpoint-500/tokenizer_config.json
Special tokens file saved in ../../../output/bert_finetune/1960s/checkpoint-500/special_tokens_map.json
/Users/Zahed001/miniforge3/envs/torch/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2339: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy.

decade: 1970s
df size : (10716, 6)
train_dataset: (8572, 2)
validation_dataset: (1072, 2)
test_dataset: (1072, 2)


...

decade: 1990s
df size : (4469, 6)
train_dataset: (3575, 2)
validation_dataset: (447, 2)
test_dataset: (447, 2)


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 78.20ba/s]
using `logging_steps` to initialize `eval_steps` to 500
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file https://huggingface.co/dbmdz/bert-base-historic-dutch-cased/resolve/main/config.json from cache at /Users/Zahed001/.cache/huggingface/transformers/1dd6939500d72963b7c7ed7033ba4857c1576a28f9f2819d8dde7a71f02e879e.5ffd597ad0cae5679a9afabdf80a381f3c67384ce1edbb6f01a13a7730a45b2e
Model config BertConfig {
  "_name_or_path": "dbmdz/bert-base-historic-dutch-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidd

Step,Training Loss,Validation Loss
500,0.786900,0.773090


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 447
  Batch size = 16
Saving model checkpoint to ../../../output/bert_finetune/1990s/checkpoint-500
Configuration saved in ../../../output/bert_finetune/1990s/checkpoint-500/config.json
Model weights saved in ../../../output/bert_finetune/1990s/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ../../../output/bert_finetune/1990s/checkpoint-500/tokenizer_config.json
Special tokens file saved in ../../../output/bert_finetune/1990s/checkpoint-500/special_tokens_map.json
/Users/Zahed001/miniforge3/envs/torch/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2339: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. 

CPU times: user 8h 28min 25s, sys: 57min 13s, total: 9h 25min 39s
Wall time: 3h 40min 54s
